### IMPORTS

In [26]:
import sys, os
sys.path.append(os.path.join(sys.path[0],'Pythons'))

In [2]:
from Carga_Descarga import *
from CRM_Central import *

In [3]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

### CONSTANTES

In [4]:
# Fechas
today = datetime.date.today()
if today.day == 1:
    end = str(today - relativedelta(days=1))
    start = str((today - relativedelta(months=2)).replace(day=1))
    start_coupon = str((today - relativedelta(months=2)).replace(day=1) - relativedelta(days=16))
else:
    start = str((today - relativedelta(months=1)).replace(day=1))
    end = str(today + relativedelta(months=1) - relativedelta(days=(today + relativedelta(months=1)).day))
    start_coupon = str((today - relativedelta(months=1)).replace(day=1) - relativedelta(days=16))

### QUERIES

In [5]:
q_tc = '''WITH orders_table AS (
    SELECT o.order_id AS order_id,
           o.order_status AS order_status,
           o.registered_date AS registered_date,
           DATETIME(o.registered_at) AS registered_at,
           c.city_name AS city,
           CASE WHEN o.payment_mode = 'TOTAL_AMOUNT' THEN 1 ELSE 0 END AS iva
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` AS c ON o.city.city_id = c.city_id
    WHERE o.registered_date BETWEEN DATE('{0}') AND CURRENT_DATE()),
    barter_creates AS (
    SELECT bc.campaign_title AS campaign_title,
           bc.id AS id,
           bc.created_at AS created_at,
           bc.discount.type AS discount_type,
           bc.discount.amount AS discount_amount,
           bc.discount.max_amount AS discount_max_amount,
           bc.rules.min_amount AS min_amount,
           bc.owner_id AS owner_id,
           DATE(bc.rules.starts_at,cn.timezone) AS starts_at,
           DATETIME(bc.rules.starts_at,cn.timezone) AS starts_at_tt,
           DATE(bc.rules.expires_at,cn.timezone) AS expires_at,
           DATETIME(bc.rules.expires_at,cn.timezone) AS expires_at_tt,
           cn.currency_id AS currency_id,
           cn.country_name AS country,
           IFNULL(u.city,'-') AS city
    FROM `peya-data-origins-pro.cl_barter.creates` AS bc
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON bc.rules.country.iso_code = cn.iso_code_2
    LEFT JOIN `peya-argentina.user_santiago_curat.city_usuarios` AS u ON bc.owner_id = u.user AND cn.country_name = u.country
    WHERE DATE(bc.created_at) BETWEEN DATE('{1}') AND CURRENT_DATE()
          AND bc.valid),
    barter_trades AS (
    SELECT DISTINCT bt.id AS id,
           bt.trade.order_id AS order_id,
           bt.trade.amount_used AS amount_used
    FROM `peya-data-origins-pro.cl_barter.trades` AS bt
    WHERE bt.trade.type = 'CONSUME'),
    barter_table AS (
    SELECT bc.campaign_title AS campaign_title,
           bc.id AS id,
           bc.created_at AS created_at,
           bc.discount_type AS discount_type,
           bc.discount_amount AS discount_amount,
           bc.discount_max_amount AS discount_max_amount,
           bc.min_amount AS min_amount,
           bc.owner_id AS owner_id,
           bc.starts_at AS starts_at,
           bc.starts_at_tt AS starts_at_tt,
           bc.expires_at AS expires_at,
           bc.expires_at_tt AS expires_at_tt,
           bc.currency_id AS currency_id,
           bc.country AS country,
           bc.city AS city,
           bt.order_id AS order_id,
           bt.amount_used AS amount_used
    FROM barter_creates AS bc
    LEFT JOIN barter_trades AS bt ON bc.id = bt.id),
    dates_table AS (
        SELECT d.date AS fecha
        FROM `peya-bi-tools-pro.il_core.dim_date` AS d
        WHERE d.date BETWEEN DATE('{0}') AND DATE('{2}'))
SELECT bt.campaign_title AS Campaign,
       IFNULL(bt.country,'-') AS Country,
       IFNULL(o.city,bt.city) AS City,
       FORMAT_DATE('%Y-%m',d.fecha) AS Month,
       IFNULL(bt.discount_type,'-') AS Discount_Type,
       CONCAT('Disc Amount: ',bt.discount_amount,' - Max Amount: ',bt.discount_max_amount,' - MOV: ',bt.min_amount) AS Variation,
       COUNT(DISTINCT bt.id) AS Sent_Vouchers,
       COUNT(o.order_id) AS Total_Orders_Vouchers,
       COUNT(DISTINCT CASE WHEN o.order_status = 'CONFIRMED' THEN o.order_id ELSE NULL END) AS Confirmed_Orders_Vouchers,
       SUM(CASE WHEN o.order_status = 'CONFIRMED' THEN bt.amount_used ELSE 0 END) AS Confirmed_Used_Amount,
       SUM(CASE WHEN o.order_status = 'CONFIRMED' THEN bt.amount_used / ce.rate_us ELSE 0 END) AS Confirmed_Used_Amount_USD,
       SUM(CASE WHEN o.order_status = 'CONFIRMED' AND o.iva = 0 THEN bt.amount_used ELSE 0 END) AS Confirmed_Used_Amount_Iva,
       SUM(CASE WHEN o.order_status = 'CONFIRMED' AND o.iva = 0 THEN bt.amount_used / ce.rate_us ELSE 0 END) AS Confirmed_Used_Amount_USD_Iva,
       AVG(CASE WHEN o.order_id IS NOT NULL THEN DATETIME_DIFF(o.registered_at,bt.starts_at_tt,HOUR) / DATETIME_DIFF(bt.expires_at_tt,starts_at_tt,HOUR) ELSE NULL END) AS Ageing,
       AVG(CASE WHEN o.order_id IS NOT NULL THEN DATETIME_DIFF(bt.expires_at_tt,starts_at_tt,HOUR) ELSE NULL END) AS Availability
FROM dates_table AS d
LEFT JOIN barter_table AS bt ON (d.fecha <= bt.expires_at OR bt.expires_at IS NULL) 
                                AND (d.fecha >= bt.starts_at OR bt.starts_at IS NULL)
LEFT JOIN orders_table AS o ON bt.order_id = o.order_id AND d.fecha = o.registered_date
LEFT JOIN `peya-bi-tools-pro.il_core.dim_currency_exchange` AS ce ON bt.currency_id = ce.currency_id AND DATE_TRUNC(o.registered_date,MONTH) = ce.currency_exchange_date
WHERE UPPER(bt.campaign_title) NOT LIKE '%%OTHER_CC_%%'
      AND UPPER(bt.campaign_title) NOT LIKE '%%OTHER_AUTOCOMP_%%'
      AND UPPER(bt.campaign_title) NOT LIKE '%%OTHER_EMPLOYEE_%%'
      AND UPPER(bt.campaign_title) NOT LIKE '%%OTHER_OTHER_%%'
      AND UPPER(bt.campaign_title) NOT LIKE '%%OTHER_NCR_%%'
      AND bt.campaign_title != ''
GROUP BY 1,2,3,4,5,6
ORDER BY 3'''.format(start,start_coupon,end)

q_stats = '''WITH barter_table AS (
    SELECT bt.trade.order_id AS order_id,
           bc.campaign_title AS campaign,
           bc.discount.type AS discount_type,
           bc.discount.amount AS discount_amount,
           bc.discount.max_amount AS discount_max_amount,
           bc.rules.min_amount AS min_amount,
           bt.trade.amount_used AS amount_used           
    FROM `peya-data-origins-pro.cl_barter.trades` AS bt 
    LEFT JOIN `peya-data-origins-pro.cl_barter.creates` AS bc ON bt.id = bc.id
    WHERE DATE(bc.created_at) BETWEEN DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -4 MONTH),MONTH) AND CURRENT_DATE()
          AND DATE(bt.tradedAt) BETWEEN DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -4 MONTH),MONTH) AND CURRENT_DATE()
          AND bt.trade.type = 'CONSUME'),
    orders_table AS (
    SELECT o.user.id AS user,
           cn.country_name AS country,
           IFNULL(c.city_name,'-') AS city,
           o.order_status AS status,
           o.order_id AS order_id,
           o.registered_date AS fecha,
           CASE WHEN bt.campaign IS NULL THEN 0 ELSE 1 END AS crm,
           CASE WHEN o.with_logistics THEN o.commission_amount + o.shipping_amount ELSE o.commission_amount END AS revenue,
           CASE WHEN o.with_logistics THEN (o.commission_amount + o.shipping_amount) / ce.rate_us ELSE o.commission_amount / ce.rate_us END AS revenue_usd,
           IFNULL(o.shipping_amount_no_discount + o.amount_no_discount,0) AS total_paid,
           IFNULL(o.shipping_amount_no_discount + o.amount_no_discount,0) / ce.rate_us AS total_paid_usd,
           IFNULL(bt.amount_used,0) AS amount_used,
           IFNULL(bt.amount_used,0) / ce.rate_us AS amount_used_usd,
           SUM(od.discount_amount) AS total_disc,
           SUM(od.discount_amount) / ce.rate_us AS total_disc_usd,
           SUM(CASE WHEN UPPER(od.discount_type_name) = 'VOUCHER' THEN od.discount_amount ELSE 0 END) AS total_voucher_disc,
           SUM(CASE WHEN UPPER(od.discount_type_name) = 'VOUCHER' THEN od.discount_amount ELSE 0 END) / ce.rate_us AS total_voucher_disc_usd
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o,
    UNNEST(discounts) AS od
    LEFT JOIN barter_table AS bt ON o.order_id = bt.order_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` AS c ON o.city.city_id = c.city_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_currency_exchange` AS ce ON cn.currency_id = ce.currency_id AND DATE_TRUNC(o.registered_date,MONTH) = ce.currency_exchange_date
    WHERE o.registered_date BETWEEN DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -3 MONTH),MONTH) AND CURRENT_DATE()
    GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,ce.rate_us),
    acq_table AS (
    SELECT ot.user AS user,
           ot.country AS country,
           ot.city AS city,
           ot.fecha AS fecha,
           bt.campaign AS campaign,
           bt.discount_type AS discount_type,
           bt.discount_amount AS discount_amount,
           bt.discount_max_amount AS discount_max_amount,
           bt.min_amount AS min_amount,
           SUM(bt.amount_used) AS amount_used,
           SUM(ot.amount_used_usd) AS amount_used_usd
    FROM orders_table AS ot
    LEFT JOIN barter_table AS bt ON ot.order_id = bt.order_id
    WHERE bt.campaign IS NOT NULL
    GROUP BY 1,2,3,4,5,6,7,8,9),
    cohorts_table AS (
    SELECT acq.user AS user,
           acq.country AS country,
           acq.campaign AS campaign,
           acq.city AS city,
           acq.discount_type AS discount_type,
           acq.discount_amount AS discount_amount,
           acq.discount_max_amount AS discount_max_amount,
           acq.min_amount AS min_amount,
           acq.fecha AS fecha,
           acq.amount_used AS original_amount_used,
           acq.amount_used_usd AS original_amount_used_usd,
           CASE WHEN DATE_DIFF(ot.fecha,acq.fecha,DAY) = 0 THEN 'ORIGINALS'
                WHEN DATE_DIFF(ot.fecha,acq.fecha,DAY) BETWEEN 1 AND 30 THEN 'M1'
                WHEN DATE_DIFF(ot.fecha,acq.fecha,DAY) BETWEEN 31 AND 60 THEN 'M2'
                WHEN DATE_DIFF(ot.fecha,acq.fecha,DAY) BETWEEN 61 AND 90 THEN 'M3' END AS cohort,
           IFNULL(COUNT(DISTINCT CASE WHEN ot.status != 'CONFIRMED' THEN NULL ELSE ot.order_id END),0) AS orders,
           SUM(CASE WHEN ot.status != 'CONFIRMED' THEN 0 ELSE ot.revenue END) AS revenue,
           SUM(CASE WHEN ot.status != 'CONFIRMED' THEN 0 ELSE ot.revenue_usd END) AS revenue_usd,
           SUM(CASE WHEN ot.status != 'CONFIRMED' THEN 0 ELSE ot.total_paid END) AS total_paid,
           SUM(CASE WHEN ot.status != 'CONFIRMED' THEN 0 ELSE ot.total_paid_usd END) AS total_paid_usd,
           SUM(CASE WHEN ot.status != 'CONFIRMED' THEN 0 ELSE ot.total_disc END) AS total_disc,
           SUM(CASE WHEN ot.status != 'CONFIRMED' THEN 0 ELSE ot.total_disc_usd END) AS total_disc_usd,
           SUM(CASE WHEN ot.status != 'CONFIRMED' THEN 0 ELSE ot.total_voucher_disc END) AS total_voucher_disc,
           SUM(CASE WHEN ot.status != 'CONFIRMED' THEN 0 ELSE ot.total_voucher_disc_usd END) AS total_voucher_disc_usd,
           SUM(CASE WHEN ot.status != 'CONFIRMED' THEN 0 ELSE ot.amount_used END) AS amount_used,
           SUM(CASE WHEN ot.status != 'CONFIRMED' THEN 0 ELSE ot.amount_used_usd END) AS amount_used_usd
    FROM acq_table AS acq
    LEFT JOIN orders_table AS ot ON acq.user = ot.user
    WHERE ot.fecha BETWEEN acq.fecha AND DATE_ADD(acq.fecha, INTERVAL 180 DAY)
    GROUP BY 1,2,3,4,5,6,7,8,9,10,11,ot.fecha,ot.status)
SELECT FORMAT_DATE('%Y-%m', ct.fecha) AS Month,
       ct.country AS Country,
       ct.city AS City,
       ct.campaign AS Campaign,
       ct.discount_type AS Discount_Type,
       CONCAT('Disc Amount: ',ct.discount_amount,' - Max Amount: ',ct.discount_max_amount,' - MOV: ',ct.min_amount) AS Variation,
       SUM(ct.original_amount_used) AS Original_Amount_Used,
       SUM(ct.original_amount_used_usd) AS Original_Amount_Used_USD,
       COUNT(DISTINCT CASE WHEN ct.cohort = 'ORIGINALS' THEN ct.user ELSE NULL END) AS Acquisitions,
       COUNT(DISTINCT CASE WHEN ct.cohort = 'M1' AND ct.orders > 0 THEN ct.user ELSE NULL END) AS M1_Users,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.orders ELSE 0 END) AS M1_Orders,
       COUNT(DISTINCT CASE WHEN ct.cohort = 'M2' AND ct.orders > 0 THEN ct.user ELSE NULL END) AS M2_Users,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.orders ELSE 0 END) AS M2_Orders,
       COUNT(DISTINCT CASE WHEN ct.cohort = 'M3' AND ct.orders > 0 THEN ct.user ELSE NULL END) AS M3_Users,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.orders ELSE 0 END) AS M3_Orders,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.revenue ELSE 0 END) AS M1_Revenue,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.revenue_usd ELSE 0 END) AS M1_Revenue_USD,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.total_paid ELSE 0 END) AS M1_Total_Paid,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.total_paid_usd ELSE 0 END) AS M1_Total_Paid_USD,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.total_disc ELSE 0 END) AS M1_Total_Disc,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.total_disc_usd ELSE 0 END) AS M1_Total_Disc_USD,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.total_voucher_disc ELSE 0 END) AS M1_Total_Voucher_Disc,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.total_voucher_disc_usd ELSE 0 END) AS M1_Total_Voucher_Disc_USD,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.amount_used ELSE 0 END) AS M1_Amount_Used,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.amount_used_usd ELSE 0 END) AS M1_Amount_Used_USD,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.revenue ELSE 0 END) AS M2_Revenue,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.revenue_usd ELSE 0 END) AS M2_Revenue_USD,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.total_paid ELSE 0 END) AS M2_Total_Paid,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.total_paid_usd ELSE 0 END) AS M2_Total_Paid_USD,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.total_disc ELSE 0 END) AS M2_Total_Disc,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.total_disc_usd ELSE 0 END) AS M2_Total_Disc_USD,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.total_voucher_disc ELSE 0 END) AS M2_Total_Voucher_Disc,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.total_voucher_disc_usd ELSE 0 END) AS M2_Total_Voucher_Disc_USD,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.amount_used ELSE 0 END) AS M2_Amount_Used,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.amount_used_usd ELSE 0 END) AS M2_Amount_Used_USD,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.revenue ELSE 0 END) AS M3_Revenue,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.revenue_usd ELSE 0 END) AS M3_Revenue_USD,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.total_paid ELSE 0 END) AS M3_Total_Paid,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.total_paid_usd ELSE 0 END) AS M3_Total_Paid_USD,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.total_disc ELSE 0 END) AS M3_Total_Disc,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.total_disc_usd ELSE 0 END) AS M3_Total_Disc_USD,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.total_voucher_disc ELSE 0 END) AS M3_Total_Voucher_Disc,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.total_voucher_disc_usd ELSE 0 END) AS M3_Total_Voucher_Disc_USD,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.amount_used ELSE 0 END) AS M3_Amount_Used,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.amount_used_usd ELSE 0 END) AS M3_Amount_Used_USD,
FROM cohorts_table AS ct
WHERE UPPER(ct.campaign) NOT LIKE '%%OTHER_CC_%%'
      AND UPPER(ct.campaign) NOT LIKE '%%OTHER_AUTOCOMP_%%'
      AND UPPER(ct.campaign) NOT LIKE '%%OTHER_EMPLOYEE_%%'
      AND UPPER(ct.campaign) NOT LIKE '%%OTHER_OTHER_%%'
      AND UPPER(ct.campaign) NOT LIKE '%%OTHER_NCR_%%'
GROUP BY 1,2,3,4,5,6'''

In [6]:
# Descargo la data
bq_tc = pd.io.gbq.read_gbq(q_tc, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████| 91340/91340 [00:32<00:00, 2835.79rows/s]


In [7]:
bq_stats = pd.io.gbq.read_gbq(q_stats, project_id='peya-argentina', dialect='standard')

Downloading: 100%|█████████████████████████████████████████████████████████| 126234/126234 [02:03<00:00, 1019.41rows/s]


In [8]:
# Copio los datos
tc = bq_tc.copy()
stats = bq_stats.copy()

### TRABAJO

#### TRABAJO LOS CUPONES

In [9]:
# Doy formato a las columnas
cols_str = ['Campaign','Country','Month','City','Discount_Type','Variation']
cols_float = [i for i in tc.columns if i not in cols_str]
tc[cols_float] = tc[cols_float].astype(float)
tc[cols_str] = tc[cols_str].apply(lambda x: x.astype(str).str.upper())
# Saco los espacios de los nombres de campañas
tc['Campaign'] = tc['Campaign'].str.replace(' ', '')
# Doy formato a la fecha
tc['Month'] = pd.to_datetime(tc['Month'], format='%Y-%m').dt.strftime('%Y-%m')

In [10]:
# Creo las columnas para los segmentos
tc['Benefit'] = tc['Campaign'].apply(lambda x: filtros(x,'BENEFICIOS'))
tc['Type'] = tc['Campaign'].apply(lambda x: filtros(x,'TIPOS'))
tc['Segment'] = tc['Campaign'].apply(lambda x: filtros(x,'SEGMENTOS'))
tc['Level'] = tc['Campaign'].apply(lambda x: filtros(x,'NIVELES'))
tc['Campaña'] = tc['Campaign'].apply(lambda x: filtros(x,'CAMPAIGNS'))
tc['Automated'] = tc['Campaign'].apply(lambda x: filtros(x,'AUTOMATIONS'))
tc['Budget'] = tc['Campaign'].apply(lambda x: filtros(x,'BUDGET'))
# Coloco el IVA
tc = func_iva(tc)

In [11]:
# Saco el IVA
used = 'Confirmed_Used_Amount'
used_iva = 'Confirmed_Used_Amount_Iva'
used_usd = 'Confirmed_Used_Amount_USD'
used_usd_iva = 'Confirmed_Used_Amount_USD_Iva'
tc['Confirmed_Used_Amount_Final'] = tc[used] - tc[used_iva] + tc[used_iva] / (1 + tc['IVA']) * tc['Extra']
tc['Confirmed_Used_Amount_Final_USD'] = tc[used_iva] - tc[used_usd_iva] + tc[used_usd_iva] / (1 + tc['IVA']) * tc['Extra']

#### TRABAJO EL ADR Y COHORTS

In [12]:
# Doy formato a las columnas
cols_str = ['Campaign','Country','Month','City','Discount_Type','Variation']
cols_float = [i for i in stats.columns if i not in cols_str]
stats[cols_float] = stats[cols_float].astype(float)
stats[cols_str] = stats[cols_str].apply(lambda x: x.astype(str).str.upper())
# Saco los espacios de los nombres de campañas
stats['Campaign'] = stats['Campaign'].str.replace(' ', '')
# Doy formato a la fecha
stats['Month'] = pd.to_datetime(stats['Month'], format='%Y-%m').dt.strftime('%Y-%m')
# Ordeno la base
stats.sort_values(by=['Country','Month','Campaign','City'],inplace=True)

In [13]:
# Llevo a cabo la limpieza del Reorder
cols_final = ['M1_Users_Final','M2_Users_Final','M3_Users_Final',
              'M1_Orders_Final','M2_Orders_Final','M3_Orders_Final',
              'Acq_M1','Acq_M2','Acq_M3']

for i in cols_final:
    stats[i] = 0
# Completo las columnas que cree
cols_toFill_m1 = ['M1_Users_Final','M1_Orders_Final','Acq_M1']
cols_fill_m1 = ['M1_Users','M1_Orders','Acquisitions']
cols_toFill_m2 = ['M2_Users_Final','M2_Orders_Final','Acq_M2']
cols_fill_m2 = ['M2_Users','M2_Orders','Acquisitions']
cols_toFill_m3 = ['M3_Users_Final','M3_Orders_Final','Acq_M3']
cols_fill_m3 = ['M3_Users','M3_Orders','Acquisitions']
months = stats.sort_values(by=['Month'],ascending=False)['Month'].unique().tolist()
if len(months) > 3:
    # Para todos los meses coloco los valores en M1
    stats[cols_toFill_m1] = stats[cols_fill_m1].values
    # Para el tercer y cuarto mes coloco los valores en M2
    stats.loc[stats['Month'].isin(months[2:]),cols_toFill_m2] = stats[stats['Month'].isin(months[2:])][cols_fill_m2].values
    # Para el cuarto mes coloco los valores en M3
    stats.loc[stats['Month'] == months[-1],cols_toFill_m3] = stats[stats['Month'] == months[-1]][cols_fill_m3].values
else:
    # Para todos los meses coloco los valores en M1
    stats[cols_toFill_m1] = stats[cols_fill_m1].values
    # Para el segundo mes coloco los valores en M2
    stats.loc[stats['Month'].isin(months[1:]),cols_toFill_m2] = stats[stats['Month'].isin(months[1:])][cols_fill_m2].values
    # Para el cuarto mes coloco los valores en M3
    stats.loc[stats['Month'] == months[-1],cols_toFill_m3] = stats[stats['Month'] == months[-1]][cols_fill_m3].values

In [14]:
# Hago una PT sacando el Month
index = ['Country','City','Campaign','Discount_Type','Variation']
values = [i for i in stats.columns if i not in index+['Month']]
pt_stats = stats.pivot_table(index=index,values=values,aggfunc='sum').reset_index()

In [15]:
# Creo las columnas para los segmentos
pt_stats['Benefit'] = pt_stats['Campaign'].apply(lambda x: filtros(x,'BENEFICIOS'))
pt_stats['Type'] = pt_stats['Campaign'].apply(lambda x: filtros(x,'TIPOS'))
pt_stats['Segment'] = pt_stats['Campaign'].apply(lambda x: filtros(x,'SEGMENTOS'))
pt_stats['Level'] = pt_stats['Campaign'].apply(lambda x: filtros(x,'NIVELES'))
pt_stats['Campaña'] = pt_stats['Campaign'].apply(lambda x: filtros(x,'CAMPAIGNS'))
pt_stats['Automated'] = pt_stats['Campaign'].apply(lambda x: filtros(x,'AUTOMATIONS'))
pt_stats['Budget'] = pt_stats['Campaign'].apply(lambda x: filtros(x,'BUDGET'))

#### UNION

In [16]:
# Creo una columna extra para llevar a cabo la union
tc['Union'] = tc['Country'] + ' - ' + tc['City'] + ' - ' + tc['Campaign'] + ' - ' + tc['Discount_Type'] + ' - ' + tc['Variation']
pt_stats['Union'] = pt_stats['Country'] + ' - ' + pt_stats['City'] + ' - ' + pt_stats['Campaign'] + ' - ' + pt_stats['Discount_Type'] + ' - ' + pt_stats['Variation']
# Hago una lista de diccionarios
months = tc.sort_values(by=['Month'],ascending=False)['Month'].unique().tolist()
dicc_union = {}
for i in months:
    dicc_union[i] = tc[tc['Month'] == i]['Union'].to_list()
# Coloco el mes en Stats para saber con que mes unir las tablas y evitar repeticiones
pt_stats.loc[pt_stats['Union'].isin(dicc_union[months[0]]),'Month'] = months[0]
pt_stats.loc[(pt_stats['Union'].isin(dicc_union[months[1]]))&(pt_stats['Month'].isna()),'Month'] = months[1]
pt_stats.loc[pt_stats['Month'].isna(),'Month'] = '-'

In [17]:
# Filtro aquellas Stats que no se cruzan con nada en TC y las agrego con Month = '-'
cols = ['Campaign','Country','City','Month','Variation','Discount_Type','Union','Benefit','Type','Segment',
        'Level','Campaña','Automated','Budget']
new_cols = list(set(tc.columns) - set(cols))
insert_extra = pt_stats[pt_stats['Month'] == '-'][cols].copy()
for i in new_cols:
    insert_extra[i] = '-'
insert_extra['Month'] = '-'
tc = tc.append(insert_extra)

In [18]:
# Hago el merge de TC y Stats
cols = ['Month','Union','M1_Users_Final','M2_Users_Final','M3_Users_Final','Acq_M1','Acq_M2','Acq_M3','M1_Orders_Final',
        'M2_Orders_Final','M3_Orders_Final','M1_Total_Paid_USD','M1_Total_Disc_USD','M1_Amount_Used_USD','M1_Revenue_USD',
        'M2_Total_Paid_USD','M2_Total_Disc_USD','M2_Amount_Used_USD','M2_Revenue_USD','M3_Total_Paid_USD','M3_Total_Disc_USD',
        'M3_Amount_Used_USD','M3_Revenue_USD','Original_Amount_Used_USD']
final = tc.merge(pt_stats[cols],on=['Month','Union'],how='left').copy()
final.replace([np.nan,np.inf,-np.inf],'-',inplace=True)

In [19]:
# Filtro las columnas finales para achicar el DF
cols_drop = ['Confirmed_Used_Amount_USD','Confirmed_Used_Amount_USD_Iva','IVA','Extra',
             'Confirmed_Used_Amount_Final_USD','Union']
final.drop(cols_drop,axis=1,inplace=True)
# Ordeno las columnas
cols = ['Campaign','Month','Country','City','Discount_Type','Type','Segment','Variation','Level','Campaña','Automated','Budget']
cols_resto = [i for i in final.columns if i not in cols]
final = final[cols+cols_resto].copy()
# Filtro las campañas sin mes (sin movimientos estos ultimos 2 meses) ni Type
final = final[(final['Month'] != '-')&(final['Type'] != '-')].copy()

In [20]:
# Divido por pais
cols_sort = ['City','Campaign','Month']
final_arg = final[final['Country'] == 'ARGENTINA'].sort_values(by=cols_sort,ascending=False).copy()
final_chi = final[final['Country'] == 'CHILE'].sort_values(by=cols_sort,ascending=False).copy()
# Dropeo la columna country para achicar el DF
final_arg.drop(['Country'],axis=1,inplace=True)
final_chi.drop(['Country'],axis=1,inplace=True)

#### TIPOS DE CAMPAÑAS

In [21]:
# Creo un DF con las unique combinations
cols = ['Budget','Type','Segment','Variation','Discount_Type','Level','Campaña','Automated','Country']
campaign_type = final.groupby(cols).size().reset_index().drop([0], axis = 1)
# Ordeno el DF
cols = ['Budget','Type','Segment']
campaign_type.sort_values(by=cols,ascending=False,inplace=True)
# Divido por pais
ct_arg = campaign_type[campaign_type['Country'] == 'ARGENTINA'].copy()
ct_chi = campaign_type[campaign_type['Country'] == 'CHILE'].copy()

### CARGA

In [257]:
# Carga Argentina
carga(final_arg,'1NJtY4CpejWDyEDSBOFEdWJIVgq7Fvd_U0O6UlUBZuUk','Crudo Performance')
carga(ct_arg,'1NJtY4CpejWDyEDSBOFEdWJIVgq7Fvd_U0O6UlUBZuUk','Filtro Campaigns')

Carga Correcta!
Carga Correcta!


In [192]:
# Carga Chile
carga(final_chi,'1WrgzvPGpTIqK5GQiMEk4FkfUB7LVE21d1BI670yx_Os','Crudo Performance')
carga(ct_chi,'1WrgzvPGpTIqK5GQiMEk4FkfUB7LVE21d1BI670yx_Os','Filtro Campaigns')

Carga Correcta!
